# Modelling Cortico-Cerebellar Connectivity

## Questions:
* What is the best model of cortico-cerebellar connectivity? 
* What is the topography of connections between the cortex and the cerebellum?
    * One-to-one connection topography?

## Goals:
* Estimate a model on cortical activity patterns and predict cerebellar activity pattern on a new set of data.
* Find the most useful model of cortico-cerebellar connectivity

## Background:
### Cortico-cerebellar connectivity in non-human primates
* (Kelly & Strick, 2003)

### Cortico-cerebellar connectivity in humans
* Methods using resting state functional connectivity:
    * (Habas et al., 2009)
    * (Buckner et al., 2011)
    * (Marek et al., 2018)
* Methods using functional gradients:
    * (Tian et al., 2020)
    * (Guell et al., 2018)

### Gap:
* assuming one to one connections, the previous models of cortico-cerebellar connectivity in humans, have used a winner-take-all approach and discovered the functional organization of cerebellum. __How am I going to address this using PLS or bi-clustering?__
* cortico-cerebellar networks not assuming the one-to-one connection (with no assumptions at al)
    
### Hypothesis:
* relaxing the assumption of one-to-one connection, a connectivity model that incorporates the connection topography/organization between the two structures will perform better at prediction activity patterns of the cerebellum on new dataset. _might be good if I use some materials from introduction in the bi-clustering paper_

## Material and Methods:
* MDTB dataset:
    * two task sets, 4 sessions, 32 runs in total
* Cortical activity pattterns were used as predictors and cerebellar activity patterns were used as responses.

Re-iterating the __goal__ here: __estimate connectivity weights and connectivity pattern between the cerebellum and cortex that makes the best predictions for cerebellar activity__

** How can I investigate connection topography??

### Analysis:
Multivariate multiple regression will be used to model cerebellar activity profiles as a linear function of cortical activity profiles across a range of conditions! The regression model here is:
\begin{align*}
&Y_{n*m}\ = X_{n*p}W_{p*m} + E_{n*m}
\\
\end{align*}

Where $n$ is the number of conditions, $m$ is the number of cerebellar voxels, $p$ is the number of cortical voxels/tessels, $W$ is the matrix of connectivity weights, and $E$ is the matrix containing the residuals. $Y$ is the matrix where each column contains the activity profile of a voxel in the cerebellum (response matrix) and $X$ is the matrix where each column contains activity profile of a voxel in the neocortex (explanatory matrix, design matrix). 

### Limitations:
There is high multicollinearity between explanatory variables (activity profiles of cerebellar voxels) which results from correlations between activity profiles of cortical tessles and the fact that the number of observations (in this case the number of conditions) is lower than the number of explanatory variables. This multicollinearity makes the estimated regression coefficients highly variable and will lead to poor predictions.

### Regression models:
There are a number of approaches that try to overcome this limitation. These approaches are:
#### Step-wise Regression
Selecting a subset of explanatory variables through a stepwise regression. This subset selection method ultimately leads to a subset of explanatory variables that best explain the variance in the response variable. This method is time-consuming for the purpose of this analysis. However, it might be a good approach to find the overlaps between cerebellar targets of cortical regions. Basically, for each cerebellar voxel, we try to find the best set of explanatory variables (cortical tessels) that best predict the activity profile, create a map of prediction accuracy of models and investigate the overlap between different models (different subsets of tassels' activity profiles)
#### Ridge Regression
a shrinkage method that shrinks the regression coefficients by imposing a size penalty. 
* Ridge regression and PCA: 
    * Ridge regression shrinks the direction of small variances in the design matrix the most,
* Ridge regression as an optimization problem: It tries to optimize penalized sums of squares:
\begin{align*}
&J\ = \sum (y_i - W_0 + \sum x_{ij} W_j)^2 + \lambda \sum W_j^2
\\
\end{align*}
In matrix notation:
\begin{align*}
&J\ = (Y - XW)^T (Y - XW) + W^T W
\\
\end{align*}
    * $\lambda$ is a parameter that determines the amount of shrinkage of the parameters. The higher the , more shrinkage will be applied
    
#### Principal Component Regression (PCR)
This method tries to find a set of orthogonal variables that best explain the covariance between the explanatory variables by applying PCA to the design matrix X. Then it regresses Y on the set of these orthogonal variables. It is a two-step process and as a optimization problem: 
1. PCA to X and 
    * $X = ZV$ subject to constraint $VV^T = 1$
        - $Z$ contains score variables and $V$ is the matrix of weights that transforms $X$ into the latent space.
        - The objective function for this step is the objective function of PCA:
        \begin{align*}
        &J\ = |X - ZV^T|^2 = |X - XVV^T|^2
        \\
        \end{align*}
2. Regress $Y$ onto principal components of $X$
    * $Y = ZW + E$ with the objective function:
    \begin{align*}
    &J\ = |Y - ZW|^2 
    \\
    \end{align*}
    
Or we combine these steps together in a single step (this will give different results):
J = Y - ZW2 = Y - XVW2, subject to constraint: VTV= 1
\begin{align*}
&J\ = |Y - ZW|^2 = |Y - XVW|^2, \text{subject to constraint:}\ VV^T = 1
\\
\end{align*}

As the variables are orthogonal, the problem of multicollinearity is solved. However, like all the other methods that use PCA, a question remains: What is the best number of components? In addition to this, the components are chosen to maximize the explained variance of X, not Y. Hence, the chosen components might not be good candidates to explain the highest variance in Y. To overcome the later limitation of this method, partial least squares regression (PLS-R) is used.

#### Partial Least Squares Regression (PLS-R)
This method also tries to solve the problem of multicollinearity in the design matrix. It estimates the latent space for both $X$ and $Y$ so as to maximize the covariance between the two. Basically, it tries to project both $X$ and $Y$ onto latent spaces and it tries to maximize the covariance between the latent spaces of $X$ and $Y$.

\begin{align*}
&X_{n*p} = Z_{n*q} V_{p*q}^T + E_{n*p}
\\
\\
&Y_{n*m} = U_{n*q} Q_{m*q}^T + F_{n*m}
\\
\end{align*}
Where $n$ is the number of conditions, $m$ is the number of cerebellar voxels, $p$ is the number of cortical voxels/tessels and $q$ is the number of PLS components.
As an optimization problem:

* As an optimization problem :
    \begin{align*}
    &Z_{n*q} = X_{n*p}V_{p*q}, \text{subject to constraint:}\ VV^T = 1
    \\
    \end{align*}
    
    $V$ is the matrix of weights that transforms $X$ into a latent space ($Z$)
    
    \begin{align*}
    &U_{n*q} = Y_{n*m}C_{m*q}, \text{subject to constraint:}\ CC^T = 1
    \\
    \end{align*}
    
    $C$ is the matrix of weights that transforms $Y$ into a latent space ($U$)
    
    The objective is to find the latent variables (spaces) between which the covariance is maximized and at the same time the method tries to predict $Y$:
    
    \begin{align*}
    &cov(Z, U) = Z^TU = V^TX^TYC
    \\
    \end{align*}
    
    
* There are different implementations of the PLSR, each with different assumptions. There are both __iterative__ and __non-iterative__ methods. To my understanding the iterative method is the most common one. But there is also a non-iterative method. The difference between the results of iterative and non-iterative methods is in the __orthogonality__ of final latent variables. Briefly, using the iterative methods like __PLS1__ (for 1-D response variable), __PLS2__, and __SIMPLS__, the latent variables will be mutually orthogonal as these methods are __iterative__ and in the end of each iteration, the effect of each estimated latent variable is subtracted from the original matrices, a process called __deflation__. The deflation process guarantees mutual orthogonality of latent variables. There is, however, a non-iterative approach. The latent variables estimated using this method will not, in general, be mutually orthogonal. In any case, there is a eigenvalue problem. The iterative methods try to solve the eigenvalue problem iteratively and the non-iterative method tries to solve this eigenvalue problem in one go.
* The algorithm tries to find $T$ and $U$ __iteratively__ and then calculates the regression coefficients as $b_i= z_i^T u_i$ at each iteration of the algorithm. Eventually, we will have a diagonal matrix ($B$) where each element is calculated in each iteration. Finally:

    \begin{align*}
    &Y_{n*m} = X_{n*p}W_{pls}
    \\
    \end{align*}
    
    where $W_{pls}= V_{p*q}^T + B_{q*q} C_{q*m}^T$. Based on this formulation, $W_{pls}$ is a $p*m$ matrix
    
    
* _I need to clarify something here to better understand how the dimensions of the matrices are determined:_
    For $A_{p*q}$, The pseudoinverse of $A$ is defined as $A_{q*p}^+$. This is useful in determining the dimensions of matrix $V$ and $W_{pls}$.
    
* _Keep in mind that we have $V_{p*q}$, $V_{q*p}^T$, and based on the relationship between a matrix and its pseudoinverse: $V_{p*q}^{T+}$_ 
* $B_{q*q}$ is a diagonal matrix.
    * __Hypothesis__: _In this model, $B$ being diagonal implies that the one-to-one topography is valid. However, not restricting B to be diagonal will imply that the one-to-one topography does not necessarily hold. My hypothesis is that not restricting B to be a diagonal matrix, in other words not imposing the one-to-one connection topography, will yield better predictions. __What would this mean in terms of mathematical implementation of the algorithm to estimate the parameters and latent structures?___
    
#### Ridge regression + Convex bi-clustering
This is the naive method mentioned in (Yu et al., 2019). You can estimate the parameters first (using ridge regression for example) and then apply a bi-clustering algorithm to the parameter matrix that is estimated to reveal the grouping structure in the parameter matrix.

#### Simultaneous Parameter Learning and Bi-clustering for Multi-Response Models (Yu et al., 2019)
They are interested in accurate parameter estimation as well as understanding the bi-cluster or checkerboard structure of the parameter matrix. The regression model is stated as before:
\begin{align*}
&Y_{n*m}\ = X_{n*p}W_{p*m} + E_{n*m}
\\
\end{align*}
And they wish to discover the bi-cluster structure among features (inputs) and responses (outputs).

As an optimization problem in general: 
* the general loss function (squared loss): $L(X, Y, W) = \sum_{s = 1}^k\|Y_s - X_s W_s\|_{2}^2$
* For regularization, $l_1$ or $l_2$ norm is used

What makes this method __special__? it all goes back to the loss function! Two loss functions are defined:
###### Hard Fusion
\begin{align*}
&\min_{W} L(X, Y, W) + \lambda _{1} R(W) + \lambda _{2}[\Omega _ \omega(W) + \Omega_ {\tilde{\omega}}(W^T)]
\\
\end{align*}

which can be re-written as:
\begin{align*}
&\min_{W} \| Y - XW \|_{F}^2 + \lambda _{1} \sum_{i = 1}^k \| W_{.i} \|_{1}+ \lambda _{2}[\Omega _ \omega(W) + \Omega_ {\tilde{\omega}}(W^T)]
\\
\end{align*}

###### Soft Fusion
\begin{align*}
&\min_{W, \Gamma} \| Y - XW \|_{F}^2 + \lambda _{1} \sum_{i = 1}^k \| W_{.i} \|_{1}+ \lambda_{2} \sum_{i = 1}^k \|W_{.i} - \Gamma_{.i}\|_{2}^2+\lambda _{3}[\Omega _ \omega(W) + \Omega_ {\tilde{\omega}}(W^T)]
\\
\end{align*}



## Results:
Preliminary results:

### Model evaluation
#### Cortico-cerebellar connectivity at the subject level
1. training model using one task set (for the current result, sc1 was used for training)
    * __crossed training__: X data of one session was paired with Y data from another session. This cross-training method is used to tackle the problem stated in Buckner et al. (that the cortical areas adjacent to cerebellum seem to be having high correlations with the cerebellm, masking the correlations from other regions) 
    * condition subset selection: for the current results, all the conditions (including instructions) were used!
2. testing: for testing, dataset from a task set different from the one used in training was used (for the current results, sc2 was used for testing)
    * __crossed testing__: the procedure of creating the crossed dataset is as before (pairing Xs and Ys from different sessions).
    * Rcv (cross validated R), Rnc (non-cross-validated R), and Rp (reliability of predictions) were used for evaluation

#### Cortico-cerebellar connectivity at the group level
Different ways of calculating group average:
1. Average over subjects after within session average is calculated for each subject separately
    * Load in activity patterns (both for cortex and the cerebellum) for each subject.
    * calculate average across runs within a session for each subject
    * average over subjects
    * fit the model using group average data
    * evaluate the model using group average data

In [6]:
# import packages needed to visualize the results
import numpy as np
import pandas as pd
import evaluation  # user-defined package
import os

import glob
import nibabel as nib

from nilearn import plotting
from nilearn import surface
from nilearn import datasets
from pathlib import Path

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets       # interactive display
%config InlineBackend.figure_format = 'svg' # other available formats are: 'retina', 'png', 'jpeg', 'pdf'
# plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/nma.mplstyle")

In [7]:
# To visualize the results:
# sestting defaults for some variables
returnSubjs = np.array([2,3,4,6,8,9,10,12,14,15,17,18,19,20,21,22,24,25,26,27,28,29,30,31])
# Setting different options of the functions
roi_dict1        = {'cortex':'tesselsWB162', 'cerebellum':'grey_nan'} 
roi_dict2        = {'cortex':'tesselsWB362', 'cerebellum':'grey_nan'} 
roi_dict3        = {'cortex':'tesselsWB642', 'cerebellum':'grey_nan'} 
trainExperiment  = 1
tMode            = 'crossed'

Create dataframes to visualize results of evaluation

In [8]:
# # Create a dataframe with all the evaluation info for subject level modelling
# DF = evaluation.eval_df(sn = returnSubjs, glm = 7, models = ['l2regress', 'plsregress'], 
#                  rois = ['tesselsWB162', 'tesselsWB362', 'tesselsWB642'], 
#                  testExper = 2)

# # save the dataframe for visualizations
# DF.to_csv('df_subjects.csv')

# # Create a dataframe with all the evaluation info for group level modelling
# DF_group = evaluation.eval_df(sn = 'group', glm = 7, models = ['l2regress', 'plsregress'], 
#                  rois = ['tesselsWB162', 'tesselsWB362', 'tesselsWB642'], 
#                  testExper = 2)

# # save the dataframe for visualizations
# DF_group.to_csv('df_group.csv')


##### Evaluation results visualization #1

In [9]:
# interactive visualization of the results
@widgets.interact(level = ['group', 'subject'],
                  model = ['plsregress', 'l2regress'],
                  xname = ['162', '362', '642'], 
                  yvar  = ['Rcv', 'Rnc', 'Rp'])
def plot_model_params(level, model, xname, yvar):
    # get the corresponding dataframe
    if level == 'group':
        myDF = pd.read_csv('df_group.csv')
    elif level == 'subject':
        myDF = pd.read_csv('df_subjects.csv')
        
    # gets the section of the dataframe corresponding to the model
    dfx = myDF.loc[myDF.xname == 'tesselsWB%s'%xname]
    dfx_model = dfx.loc[dfx.model == model]
    
    # sort the dataframe based on values of params
    df2 = dfx_model.sort_values(by ='params' , ascending=False)
    
    # plot
    figure, axes = plt. subplots(nrows=1, ncols=2, figsize = (15, 5))
    plt.subplot(121)
    sns.barplot(x='model', y=yvar, hue='params', data=df2)
    plt.title('barplot of %s - %s - %s level'% (yvar, model, level))
    
    plt.subplot(122)
    sns.lineplot(x='params', y=yvar, marker='o', err_style = 'band', 
                 dashes = False, data=df2)
    plt.title('lineplot of %s - %s - %s level'% (yvar, model, level))
    figure. tight_layout(pad=3.0)

interactive(children=(Dropdown(description='level', options=('group', 'subject'), value='group'), Dropdown(des…

##### Evaluation results visualization #2

In [10]:
# interactive visualization of the results
@widgets.interact(level = ['group', 'subject'],
                  xname = ['162', '362', '642'], 
                  yvar  = ['Rcv', 'Rnc', 'Rp', 'Ry'])
def plot_model_params(level, xname, yvar):
    model = ['plsregress', 'l2regress']
    # get the corresponding dataframe
    if level == 'group':
        myDF = pd.read_csv('df_group.csv')
    elif level == 'subject':
        myDF = pd.read_csv('df_subjects.csv')
        
    # gets the section of the dataframe corresponding to the model
    dfx = myDF.loc[myDF.xname == 'tesselsWB%s'%xname]

    dfx_model0 = dfx.loc[dfx.model == model[0]]
    dfx_model1 = dfx.loc[dfx.model == model[1]]
    
    # convert to log space for l2regress
#     vals = np.log10(dfx_model1['params'].values)
#     dfx_model1['log_params'] = vals
#     display(dfx_model1)
    
    # sort the dataframe based on values of params
    df20 = dfx_model0.sort_values(by ='params' , ascending=False)
    df21 = dfx_model1.sort_values(by ='params' , ascending=False)
        
    ymin = min(np.min(df20[yvar]), np.min(df21[yvar]))
    ymax = max(np.max(df20[yvar]), np.max(df21[yvar]))
    
    
    # plot
    figure, axes = plt.subplots(nrows=1, ncols=2, figsize = (10, 5))
    plt.subplot(121)
    sns.lineplot(x='params', y=yvar, marker='o', err_style = 'band',
                 dashes = False, data=df20)
    # control y limits
    plt.ylim(ymin - 0.01, ymax+0.01)
    # set title
    plt.title('%s VS parameter - %s - %s level'%(yvar, model[0], level))

    
    plt.subplot(122)
    sns.lineplot(x='params', y=yvar, marker='o', err_style = 'band', 
                 dashes = False, data=df21) # is it using std?????
    # control y limits
    plt.ylim(ymin-0.01, ymax+0.01)
    # set title
    plt.title('%s VS parameter - %s - %s level'%(yvar, model[1], level))
    
    figure. tight_layout(pad=3.0)

interactive(children=(Dropdown(description='level', options=('group', 'subject'), value='group'), Dropdown(des…

### Cortical and Cerebellar maps
Here, I will be showing the cortical and cerebellar maps

#### Component loading maps for PLS regression with 7 PLS components
gifti files for each component loadings averaged across subjects are:

In [ ]:
# import matplotlib.image as mpimg 
# import matplotlib.pyplot as plt 


# !wget -qO 'gg_642_6_R.png' https://drive.google.com/drive/folders/1r2VSjmFGw9MUtzEZmTrYmWOghRDq_MHd/

# # importing matplotlib modules 
# import matplotlib.image as mpimg 
# import matplotlib.pyplot as plt 
  
# # Read Images 
# # plot
# figure, axes = plt.subplots(nrows=2, ncols=2, figsize = (10, 5))
# plt.subplot(121)
# img = mpimg.imread(os.path.join('./preliminaryIMs/PLS', 'gg_162_0_L.png'))


# # Output Images 
# plt.imshow(img)

# plt.subplot(122)
# img = mpimg.imread(os.path.join('./preliminaryIMs/PLS', 'gg_162_0_R.png'))

# plt.subplot(211)
# img = mpimg.imread(os.path.join('./preliminaryIMs/PLS', 'gg_162_0_cereb.png'))

# # Output Images 
# plt.imshow(img)

#### bi-clustering of connectivity weights estimated using ridge regression with $\lambda = 250$
Results from applying spectral bi-clustering to the connectivity matrix averaged across subjects:

# Comments:

## Model evaluation:

1. For the l2regress, use log and plot in log space
2. Think about noise ceilings: Assume there were only noise in y, then also X. What is the noise ceiling?
3. For the subject level, what is seaborn using to plot the error bars?
4. is the Rcv for 7 __significantly__ higher than the other ones?
5. For both PLS and l2regress, have two extremes for the parameters. For example for PLS, start from 1 comp and go a little bit further!

### Noise ceilings:
The modeling process is a two-step process:
First, the activations are estimated 
Second, the activity profiles for cortical parcels are used as predictors (features/regressors) to predict cerebellar activity profiles.

Assuming that the estimated activations are true activations, what is the best performance we can get?

The noise ceilings can be calculated at two levels:
1. at the subject level, for models estimated for each subject: at the individual level, we have runs. We know that the model estimated for each run will be different from a model estimated from another run. To find the high noise ceiling, answer this question: assuming that the "correct" model (the model that gives rise to the data) is used, what is limiting the performance of the model? Then to calculate the higher and lower noise ceiling for each subject, you can take on an approach similar to the approach used for calculating the noise ceilings at the group level (see below). Besides, you can use the two sessions of data for testing separately (two different test sets) and then calculate the correlation between the predictions from the two sets ( __split-half noise ceiling__ ). 
2. at the group level:
    * __Highe noise ceiling__: For each subject, the correlation is calculated between the weights estimated for that subject with the average of all the subjects. The final noise ceiling will then be the calculated correlations averaged over all the subjects. No model can outperform this noise ceiling model.
    * __Low noise ceiling__: which shows how similar ( _consistant_ ) the estimated connectivity weights are across subjects. it can be the correlation between each subject's connectivity weights with all the other subjects! The overal lower noise ceiling is then the average of the correlation values across subjects.
        * a low lower noise ceiling (near to zero or less than zero) would mean that there is essentially no similarity between the subjects
        * a high lower noise ceiling indicates that connectivity weights are similar across subjects. 
        * if a model outperforms the lower noise ceiling, then on average, each subject's connectivity weights are more similar to the model than to the average connectivity weight of all the other subjects. (the model is better than just the average)
            
    
## Component Loading maps for PLS:

1. For PLS, plot comp1 vs comp2 or some other component! Might help in interpretation.
2. thnk of different ways of interpreting the component loadings
3. for PLS 7, it seems that the first component is just the average activation across all the tasks.
4. for PLS 7, there seem to be a laterality in the cortical and cerebellar maps (more in the cerebellar maps) and this laterality seem to be visible in higher ordered components (the components that are explaining less covariance)